In [1]:
import polars as pl

In [2]:
df = pl.read_ndjson('./experiments/base_updated2/log-913.ndjson', infer_schema_length=None)

In [3]:
df_schema = df.schema

In [96]:
df = pl.scan_ndjson('./experiments/base/log-9*.ndjson', schema=df_schema).collect()

#df = df.filter(pl.col('symmetric') == True)
#df = df.filter(pl.col('comparison_category') == 'riod_pooled_citest')
#df = df.filter(pl.col('llf_neg_prob_fix') == 'clipping')
df = df.filter(pl.col('num_samples') >= 100)
#df = df.filter(pl.col('alpha') == 0.01)
len(df)

2844

In [97]:
_df = df
_df = _df.with_columns(accuracy=pl.col('tp') + pl.col('tn'))

_df = _df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(0,3))

grouping_keys = ['comparison_type', 'num_samples', 'alpha', 'tikhonov_lambda', 'comparison_category', 'llf_neg_prob_fix']
by_keys = ['name', 'num_clients']
y_key = 'accuracy'

_df.plot.box(y=y_key,
             by=by_keys,
             groupby=grouping_keys,
             c=by_keys[0],
             rot=60,
             cmap='Pastel2',
             ylim=(-0.1,1.1),
             legend=False,
             widget_location='bottom'
             )

BokehModel(combine_events=True, render_bundle={'docs_json': {'c9f7ee95-d6d3-4db0-b1ae-c88b15157ba2': {'version…

In [98]:
__df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'symmetric', 'num_clients', 'num_samples', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
    
__df.sort('num_samples').plot.line(x='num_samples', y='p_value_correlation',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(0.8,1.001))

bokeh backend could not plot any Elements in the Overlay.

:DynamicMap   [comparison_type,num_clients,alpha,tikhonov_lambda,llf_neg_prob_fix]
   :NdOverlay   [name]

In [99]:
__df = df

__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))

#_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))


__df = __df.explode('predicted_p_vals', 'true_p_vals')
__df = __df.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                                               (pl.col('true_p_vals') > pl.col('alpha'))) |
                                                               ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                                               (pl.col('true_p_vals') < pl.col('alpha')))
                                                               )

grouping_cols = ['num_samples', 'name', 'comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix']

__df = __df.group_by(grouping_cols).agg(pl.col('decision_agreement').mean())

__df.sort('num_samples').plot.line(x='num_samples', y='decision_agreement',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(0.8,1.001))

bokeh backend could not plot any Elements in the Overlay.

:DynamicMap   [comparison_type,num_clients,symmetric,alpha,tikhonov_lambda,llf_neg_prob_fix]
   :NdOverlay   [name]

In [92]:
_df_decision_agreements = _df

#_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(-0.1,1.1))


_df_decision_agreements = _df_decision_agreements.with_columns(accuracy=pl.col('tn')+pl.col('tp'))

grouping_cols = ['num_samples', 'name', 'comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix']

_df_decision_agreements = _df_decision_agreements.group_by(grouping_cols).agg(pl.col('accuracy').mean())

_df_decision_agreements.sort('num_samples').plot.line(x='num_samples', y='accuracy',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'], ylim=(0.8,1.001))

bokeh backend could not plot any Elements in the Overlay.

:DynamicMap   [comparison_type,num_clients,symmetric,alpha,tikhonov_lambda,llf_neg_prob_fix]
   :NdOverlay   [name]

In [93]:
_df.explode('predicted_p_vals', 'true_p_vals').plot.scatter(x='predicted_p_vals',
                                                             y='true_p_vals',
                                                             groupby=['name', 'num_samples', 'num_clients', 'llf_neg_prob_fix'],
                                                             ylim=(-0.1,1.1),
                                                             xlim=(-0.1,1.1),
                                                             width=400,
                                                             height=400,
                                                             alpha=0.5
                                                             )

BokehModel(combine_events=True, render_bundle={'docs_json': {'93b8e207-c7da-4c81-a883-dfd6a9495181': {'version…

In [95]:
__df = _df
__df = __df.explode('predicted_p_vals', 'true_p_vals')
__df = __df.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                            (pl.col('true_p_vals') > pl.col('alpha'))) |
                                            ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                            (pl.col('true_p_vals') < pl.col('alpha')))
                                            )


print(__df['decision_agreement'].value_counts())
print(__df.filter(~pl.col('decision_agreement')).group_by('name').len().sort('len', descending=True))
print(__df.filter(~pl.col('decision_agreement')).group_by('num_clients').len().sort('len', descending=True))
print(__df.filter(~pl.col('decision_agreement')).group_by('name', 'num_samples', 'num_clients').len().sort('len', descending=True))

__df.filter(~pl.col('decision_agreement')).plot.scatter(x='predicted_p_vals',
                y='true_p_vals',
                #by='decision_agreement',
                #by=['name', 'num_samples', 'num_clients'],
                groupby=['name', 'num_samples', 'num_clients'],#, 'llf_neg_prob_fix', 'alpha'],
                ylim=(-0.1,1.1),
                xlim=(-0.1,1.1),
                width=400,
                height=400,
                alpha=0.5
                )

shape: (2, 2)
┌────────────────────┬───────┐
│ decision_agreement ┆ count │
│ ---                ┆ ---   │
│ bool               ┆ u32   │
╞════════════════════╪═══════╡
│ false              ┆ 51    │
│ true               ┆ 9471  │
└────────────────────┴───────┘
shape: (7, 2)
┌─────────────────┬─────┐
│ name            ┆ len │
│ ---             ┆ --- │
│ str             ┆ u32 │
╞═════════════════╪═════╡
│ L-M Con. Indep. ┆ 15  │
│ L-B Con. Dep.   ┆ 13  │
│ L-M Con. Dep.   ┆ 10  │
│ L-B Con. Indep. ┆ 7   │
│ L-M Unc. Dep.   ┆ 3   │
│ L-B Unc. Indep. ┆ 2   │
│ L-B Unc. Dep.   ┆ 1   │
└─────────────────┴─────┘
shape: (3, 2)
┌─────────────┬─────┐
│ num_clients ┆ len │
│ ---         ┆ --- │
│ i64         ┆ u32 │
╞═════════════╪═════╡
│ 5           ┆ 24  │
│ 3           ┆ 21  │
│ 1           ┆ 6   │
└─────────────┴─────┘
shape: (38, 4)
┌─────────────────┬─────────────┬─────────────┬─────┐
│ name            ┆ num_samples ┆ num_clients ┆ len │
│ ---             ┆ ---         ┆ ---         ┆ ---

BokehModel(combine_events=True, render_bundle={'docs_json': {'535e0284-1b8a-4c20-a72c-fa2e162b20c0': {'version…

In [26]:
# L-B Con. Indep. !!!
# L-M Con. Indep. !!!
# L-M Con. Dep.

In [78]:
df = pl.scan_ndjson('./experiments/base/log-9*.ndjson', schema=df_schema).collect()

df = df.filter(pl.col('num_samples') >= 100)
#df = df.filter(pl.col('alpha') == 0.01)

len(df)

2646

In [79]:
_df = df
_df = _df.explode('predicted_p_vals', 'true_p_vals')
_df = _df.with_columns(decision_agreement=((pl.col('predicted_p_vals') > pl.col('alpha')) &
                                            (pl.col('true_p_vals') > pl.col('alpha'))) |
                                            ((pl.col('predicted_p_vals') < pl.col('alpha')) &
                                            (pl.col('true_p_vals') < pl.col('alpha')))
                                            )

_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(4))


_df = _df.sort('num_samples')


_df.plot.line(x='num_samples',
                y='decision_agreement',
                alpha=0.8,
                by='comparison_variables',
                groupby=['comparison_type','num_clients', 'alpha'],
                ylim=(-0.1,1.001)) #* _df.plot.errorbars(y='decision_agreement', yerr1='std')

BokehModel(combine_events=True, render_bundle={'docs_json': {'bb5e5fb1-f49c-4f69-80c2-57d764503262': {'version…

In [80]:
_df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'num_clients', 'num_samples', 'alpha') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(4))
    
_df.sort('num_samples').plot.line(x='num_samples',
                                  y='p_value_correlation',
                                  alpha=0.6,
                                  by='comparison_variables',
                                  groupby=['comparison_type','num_clients', 'alpha'],
                                  ylim=(0.8,1.001))

BokehModel(combine_events=True, render_bundle={'docs_json': {'b0385479-81a3-434f-a3e5-be0977936055': {'version…

In [ ]:
_df = df
_df = _df.explode('predicted_p_vals', 'true_p_vals')


_df = _df.with_columns(comparison_variables=pl.col('name').str.slice(0,3))
_df = _df.with_columns(comparison_type=pl.col('name').str.slice(4))

#_df = _df.with_columns(pl.col(''))


_df.sort('num_samples').plot.line(x='num_samples',
                                   y='decision_agreement',
                                   alpha=0.8,
                                   by='comparison_variables',
                                   groupby=['comparison_type','num_clients', 'alpha'],
                                   ylim=(-0.01,1.001))

In [ ]:
# PLOTS FROM PAPER

In [ ]:
__df = df.select('name', 'num_clients', 'num_samples', 'symmetric', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix', 'predicted_p_vals', 'true_p_vals') \
    .explode('predicted_p_vals', 'true_p_vals') \
    .group_by('name', 'symmetric', 'num_clients', 'num_samples', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix') \
    .agg(pl.corr('predicted_p_vals', 'true_p_vals')) \
    .rename({'predicted_p_vals': 'p_value_correlation'}) 
    
__df = __df.with_columns(comparison_type=pl.col('name').str.slice(0,3))
    
__df.sort('num_samples').plot.line(x='num_samples', y='p_value_correlation',alpha=0.6, by='name', groupby=['comparison_type','num_clients', 'alpha', 'tikhonov_lambda', 'llf_neg_prob_fix'])

In [78]:
import glob
import polars as pl

In [80]:
# for file in glob.glob('./experiments/base/*.ndjson'):
#     #print(file)
#     df = pl.read_ndjson(file)
#     #print(len(df))
#     df = df.filter(pl.col('num_samples') >= 100)
#     #print(len(df))
#     #break
#     df = df.write_ndjson(file)